Compute Voronoi Area for Visualizations Below

In [11]:
import numpy as np
import os
import pandas as pd
from scipy.spatial import Voronoi, voronoi_plot_2d
from shapely.geometry import Polygon
from shapely.ops import unary_union
import mplcursors
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
from shapely.geometry import Point

In [2]:
!pip install mplcursors

In [5]:
players = pd.read_csv("players.csv")
plays = pd.read_csv("plays.csv")
playerplays = pd.read_csv("player_play.csv")

# Updated with correct tracking week, purposefully did one at a time to save memory space
df_tracking= pd.read_csv('tracking_week_9.csv')

In [6]:
test = df_tracking.merge(players, on="nflId")

# Add in example gameId and playId

samplePlay = test[
    (test["gameId"] == 2022110609) &
    (test["playId"] ==  105) &
    (test["position"].isin(['WR', 'TE', 'RB']))
]

In [7]:
club_colors = {
    'ARI': '#97233F', 'ATL': '#000000', 'BAL': '#241773',
    'BUF': '#00338D', 'CAR': '#0085CA', 'CHI': '#C83803',
    'CIN': '#FB4F14', 'CLE': '#311D00', 'DAL': '#003594',
    'DEN': '#FB4F14', 'DET': '#FB4F14', 'GB': '#203731',
    'HOU': '#03202F', 'IND': '#002C5F', 'JAX': '#101820',
    'KC': '#E31837', 'LV': '#A5ACAF', 'LAC': '#0080C6',
    'LAR': '#003594', 'MIA': '#008E97', 'MIN': '#4F2683',
    'NE': '#002244', 'NO': '#D3BC8D', 'NYG': '#0B2265',
    'NYJ': '#125740', 'PHI': '#004C54', 'PIT': '#FFB612',
    'SF': '#AA0000', 'SEA': '#002244', 'TB': '#D50A0A',
    'TEN': '#0C2340', 'WAS': '#5A1414'
}

In [8]:
#Compute Voronoi Areas for plotting

def compute_voronoi_areas(samplePlay):
    samplePlay = pd.merge(
        samplePlay,
        plays[['gameId', 'playId', 'possessionTeam', 'yardlineSide', 'yardlineNumber', 'absoluteYardlineNumber']],
        how='left', on=['gameId', 'playId']
    )

    player_points = samplePlay[['nflId', 'x', 'y', 'position', 'frameId', 'playId', 'displayName_x']]
    points = player_points[['x', 'y']].values

    # Define field boundaries
    x_min, x_max = 0, 120
    y_min, y_max = 0, 53.3
    boundary_polygon = Polygon([(x_min, y_min), (x_min, y_max), (x_max, y_max), (x_max, y_min)])

    boundary_points = np.array([[x_min, y_min], [x_min, y_max], [x_max, y_min], [x_max, y_max]])
    all_points = np.vstack([points, boundary_points])

   
    vor = Voronoi(all_points)

    # Clip Voronoi regions to field boundary
    clipped_areas = []
    for i, player_row in player_points.iterrows():
        region = vor.point_region[i]
        region_vertices = vor.regions[region]
        if -1 in region_vertices or not region_vertices:
            continue  
        polygon = Polygon([vor.vertices[i] for i in region_vertices])
        clipped_polygon = polygon.intersection(boundary_polygon)
        if not clipped_polygon.is_empty:
            clipped_areas.append({
                'nflId': player_row['nflId'],
                'frameId': player_row['frameId'],
                'playId': player_row['playId'],
                'position': player_row['position'],
                'voronoiArea': clipped_polygon.area,
                'DisplayName': player_row['displayName_x']
            })

    return pd.DataFrame(clipped_areas)

In [ ]:
df  = compute_voronoi_areas(samplePlay)
df

Visualize Line Graph of Voronoi Area over Time

In [14]:
# Plot Voronoi Areas over time

def plot_separation_animation_browser(separation_data, output_file="separation_animation.html", start_frame=10, ball_snapped=None, ball_thrown=None):
    # Filter data to start from the desired frame
    filtered_data = separation_data[separation_data['frameId'] >= start_frame]
    
    pivot_data = filtered_data.pivot(index='frameId', columns='DisplayName', values='voronoiArea')
    players = pivot_data.columns

    fig, ax = plt.subplots()
    lines = {player: ax.plot([], [], label=f" {player}")[0] for player in players}

    ax.set_xlim(filtered_data['frameId'].min(), filtered_data['frameId'].max())
    ax.set_xlabel("Frame")
    ax.set_ylabel("Voronoi Area (yards)")
    ax.set_title("Player Separation Over Time")
    ax.legend(loc='upper left')
    ax.set_ylim(0, 50) 

    # Add vertical dotted line for ball snap and ball throw
    if ball_snapped is not None and ball_snapped >= start_frame:
        ax.axvline(x=ball_snapped, color='blue', linestyle='--', label="Ball is Snapped")
    
    if ball_thrown is not None and ball_thrown  >= start_frame:
        ax.axvline(x=ball_thrown , color='green', linestyle='--', label="Ball is Thrown")


    def update(frame):
        for player in players:
            if frame in pivot_data.index:
                lines[player].set_data(pivot_data.index[:frame - start_frame], pivot_data[player].iloc[:frame - start_frame])
        return lines.values()

   
    ani = FuncAnimation(fig, update, frames=len(pivot_data), blit=False, interval=200)
    
    ax.set_xlim(filtered_data['frameId'].min(), filtered_data['frameId'].max())
     
    #Plot in Browser

    from matplotlib.animation import HTMLWriter
    writer = HTMLWriter()
    ani.save(output_file, writer=writer)

    print(f"Animation saved as {output_file}")

    plt.close(fig)


In [15]:
plot_separation_animation_browser(df, start_frame=10, ball_snapped=136, ball_thrown=155)

Animation saved as separation_animation.html


Visualize Animated Voronoi Areas

In [16]:
import matplotlib.animation as animation
from matplotlib.patches import FancyArrowPatch
from matplotlib import cm
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
from scipy.spatial import Voronoi
from shapely.geometry import Polygon
import numpy as np
import pandas as pd
import webbrowser
import os

In [18]:
weekOneTracking = pd.read_csv("tracking_week_1.csv")
weekTwoTracking = pd.read_csv("tracking_week_2.csv")
weekThreeTracking = pd.read_csv("tracking_week_3.csv")
weekFourTracking = pd.read_csv("tracking_week_4.csv")
weekFiveTracking = pd.read_csv("tracking_week_5.csv")
weekSixTracking = pd.read_csv("tracking_week_6.csv")
weekSevenTracking = pd.read_csv("tracking_week_7.csv")
weekEightTracking = pd.read_csv("tracking_week_8.csv")
weekNineTracking = pd.read_csv("tracking_week_9.csv")

In [19]:
# Animated Voronoi Areas


weekOneTracking = pd.merge(
    weekOneTracking,
    players[['nflId','position']],
    how = "left",
    on = "nflId"
)
weekTwoTracking = pd.merge(
    weekTwoTracking,
    players[['nflId','position']],
    how = "left",
    left_on=['nflId'],
    right_on=['nflId']
)
weekThreeTracking = pd.merge(
    weekThreeTracking,
    players[['nflId','position']],
    how = "left",
    left_on=['nflId'],
    right_on=['nflId']
)
weekFourTracking = pd.merge(
    weekFourTracking,
    players[['nflId','position']],
    how = "left",
    left_on=['nflId'],
    right_on=['nflId']
)
weekFiveTracking = pd.merge(
    weekFiveTracking,
    players[['nflId','position']],
    how = "left",
    left_on=['nflId'],
    right_on=['nflId']
)
weekSixTracking = pd.merge(
    weekSixTracking,
    players[['nflId','position']],
    how = "left",
    left_on=['nflId'],
    right_on=['nflId']
)
weekSevenTracking = pd.merge(
    weekSevenTracking,
    players[['nflId','position']],
    how = "left",
    left_on=['nflId'],
    right_on=['nflId']
)
weekEightTracking = pd.merge(
    weekEightTracking,
    players[['nflId','position']],
    how = "left",
    left_on=['nflId'],
    right_on=['nflId']
)
weekNineTracking = pd.merge(
    weekNineTracking,
    players[['nflId','position']],
    how = "left",
    left_on=['nflId'],
    right_on=['nflId']
)

def voronoi_visual_single_play(game, play, club_colors=None):
    #Change which week the play is in
    samplePlay = weekNineTracking[(weekNineTracking['gameId'] == game)&(weekNineTracking['playId'] == play)]
    samplePlay = pd.merge(
        samplePlay,
        plays[['gameId', 'playId', 'possessionTeam', 'yardlineSide', 'yardlineNumber', 'absoluteYardlineNumber', 'yardsToGo']],
        how='left',
        left_on=['gameId', 'playId'],
        right_on=['gameId', 'playId']
    )
    samplePlayPoints = samplePlay[['nflId', 'x', 'y', 'club', 'position', 'possessionTeam', 'yardlineSide', 'yardlineNumber', 
                                   'absoluteYardlineNumber', 'frameId', 'playId', 'yardsToGo']]
    samplePlayPoints['playDirection'] = None

    for index, row in samplePlayPoints.iterrows():
        if row['possessionTeam'] == row['yardlineSide']:
            if row['absoluteYardlineNumber'] > 60:
                samplePlayPoints.at[index, 'playDirection'] = "left"
            else:
                samplePlayPoints.at[index, 'playDirection'] = "right"
        else:
            if row['absoluteYardlineNumber'] > 60:
                samplePlayPoints.at[index, 'playDirection'] = "right"
            else:
                samplePlayPoints.at[index, 'playDirection'] = "left"

    absolute_yd_line = samplePlayPoints.absoluteYardlineNumber.values[0]
    play_going_right = (samplePlayPoints.playDirection.values[0] == "right")

    line_of_scrimmage = absolute_yd_line
    first_down_marker = (
        (line_of_scrimmage + samplePlayPoints.yardsToGo.values[0])
        if play_going_right
        else (line_of_scrimmage - samplePlayPoints.yardsToGo.values[0])
    )

    grouped_data = samplePlayPoints.groupby('frameId')

    fig, ax = plt.subplots(figsize=(16, 6))

    # Club colors
    club_colors = {
        'ARI': '#97233F', 'ATL': '#000000', 'BAL': '#241773',
        'BUF': '#00338D', 'CAR': '#0085CA', 'CHI': '#C83803',
        'CIN': '#FB4F14', 'CLE': '#311D00', 'DAL': '#003594',
        'DEN': '#FB4F14', 'DET': '#FB4F14', 'GB': '#203731',
        'HOU': '#03202F', 'IND': '#002C5F', 'JAX': '#101820',
        'KC': '#E31837', 'LV': '#A5ACAF', 'LAC': '#0080C6',
        'LA': '#003594', 'MIA': '#008E97', 'MIN': '#4F2683',
        'NE': '#002244', 'NO': '#D3BC8D', 'NYG': '#0B2265',
        'NYJ': '#125740', 'PHI': '#004C54', 'PIT': '#FFB612',
        'SF': '#AA0000', 'SEA': '#002244', 'TB': '#D50A0A',
        'TEN': '#0C2340', 'WAS': '#5A1414'
    }

    def update(frame_id):
        ax.clear()
        new_rows = []
        frame_data = grouped_data.get_group(frame_id)
        for _, row in frame_data.iterrows():  # Iterate over each row in the frame_data DataFrame
                    if row['position'] in ['WR', 'TE', 'RB']:
                        if row['possessionTeam'] == row['yardlineSide']:
                            if row['absoluteYardlineNumber'] < 60:
                                xpoint = row['x'] - 5
                            else:
                                xpoint = row['x'] + 5
                        else:
                            if row['absoluteYardlineNumber'] > 60:
                                xpoint = row['x'] - 5
                            else:
                                xpoint = row['x'] + 5
                        new_row = {
                            'x': xpoint,
                            'y': row['y'],
                            'position': '',
                            'club': None,
                            'nflId': 0
                        }
                        new_rows.append(new_row)
        new_rows_df = pd.DataFrame(new_rows)
        frame_data = pd.concat([frame_data, new_rows_df], ignore_index=True)
        frame_data1 = frame_data[frame_data['nflId'].notna()]
        points = frame_data1[['x', 'y']].values
        x_min, x_max = 0, 120
        y_min, y_max = 0, 53.3
        boundary_points = np.array([
            [x_min, y_min],
            [x_min, y_max],
            [x_max, y_min],
            [x_max, y_max]
        ])
        all_points = np.vstack([points, boundary_points])
        vor = Voronoi(all_points)
        boundary_polygon = Polygon([(x_min, y_min), (x_min, y_max), (x_max, y_max), (x_max, y_min)])

        clipped_polygons = []
        for region in vor.regions:
            if not -1 in region and len(region) > 0:
                polygon = Polygon([vor.vertices[i] for i in region])
                clipped_polygon = polygon.intersection(boundary_polygon)
                if not clipped_polygon.is_empty:
                    clipped_polygons.append(clipped_polygon)

        for poly in clipped_polygons:
            x, y = poly.exterior.xy
            ax.fill(x, y, edgecolor='black', facecolor='none')

        for club in frame_data['club'].unique():
            club_points = frame_data[frame_data['club'] == club]
            club_color = club_colors.get(club, 'brown')
            ax.scatter(club_points['x'], club_points['y'], label=club, s=50, color=club_color, zorder=10)

        ax.set_xlim(0, 120)
        ax.set_ylim(0, 53.3)
        ax.set_facecolor('#00C04B')
        ax.set_xticks([])
        ax.set_yticks([])
        ax.plot([first_down_marker, first_down_marker], [0, 53], color="yellow", linestyle="--", linewidth=1.5)
        ax.plot([line_of_scrimmage, line_of_scrimmage], [0, 53], color="blue", linestyle="--", linewidth=1.5)
        
        for yardline in range(10,115,5):
            ax.plot([yardline, yardline], [0, 53], color="white", linewidth=0.5, zorder=-1)
        
        for yardline in range(20,110,10):    
            for y_val in [3, 50]:
                if yardline > 60:
                    yard = 120-yardline
                else:
                    yard = yardline
                ax.text(
                    yardline, y_val, str(yard-10),
                    fontsize=26, fontfamily="Courier New", color="white", ha="center", va="center"
                )

        for y_val in [0, 53]:
            ax.text(
                first_down_marker, y_val, str('1st'),
                fontsize=16, fontfamily="Courier New", color="black", ha="center", va="center",
                bbox=dict(facecolor="#ff7f0e", edgecolor="black", linewidth=0.6)
            )

        ax.legend(loc='upper left', bbox_to_anchor=(1.05, 1), borderaxespad=0.)
        ax.set_title(f"Frame {frame_id}")

    # Create animation
    frames = list(grouped_data.groups.keys())
    ani = FuncAnimation(fig, update, frames=frames, interval=195)

    # Save to HTML
    output_path = "voronoi_animation.html"
    ani.save(output_path, writer='html')

    # Open in browser
    webbrowser.open(f"file://{os.path.abspath(output_path)}")

Visualize Full Plays

In [ ]:
import os 
import numpy as np 
import pandas as pd 
import seaborn as sns 
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import plotly.io as pio

In [ ]:
df_plays = pd.read_csv('plays.csv')
players = pd.read_csv('players.csv')
playerplay = pd.read_csv('player_play.csv')
df_tracking= pd.read_csv('tracking_week_9.csv')

In [ ]:
# Visualize full individual plays

def visualize(game_id,play_id):


    pio.renderers.default = (
        "browser"  
    )

    colors = {
        "ARI": "#97233F",
        "ATL": "#A71930",
        "BAL": "#241773",
        "BUF": "#00338D",
        "CAR": "#0085CA",
        "CHI": "#C83803",
        "CIN": "#FB4F14",
        "CLE": "#311D00",
        "DAL": "#003594",
        "DEN": "#FB4F14",
        "DET": "#0076B6",
        "GB": "#203731",
        "HOU": "#03202F",
        "IND": "#002C5F",
        "JAX": "#9F792C",
        "KC": "#E31837",
        "LA": "#FFA300",
        "LAC": "#0080C6",
        "LV": "#000000",
        "MIA": "#008E97",
        "MIN": "#4F2683",
        "NE": "#002244",
        "NO": "#D3BC8D",
        "NYG": "#0B2265",
        "NYJ": "#125740",
        "PHI": "#004C54",
        "PIT": "#FFB612",
        "SEA": "#69BE28",
        "SF": "#AA0000",
        "TB": "#D50A0A",
        "TEN": "#4B92DB",
        "WAS": "#5A1414",
        "football": "#CBB67C",
        "tackle": "#FFC0CB",
    }

    # Handle Data I/O


    df_full_tracking = df_tracking.merge(df_plays, on=["gameId", "playId"])

    df_focused = df_full_tracking[
        (df_full_tracking["playId"] == play_id) & (df_full_tracking["gameId"] == game_id)
    ]

    # Get General Play Information
    absolute_yd_line = df_focused.absoluteYardlineNumber.values[0]
    play_going_right = (
        df_focused.playDirection.values[0] == "right"
    )  # 0 if left, 1 if right

    line_of_scrimmage = absolute_yd_line

    # place LOS depending on play direction and absolute_yd_line. 110 because absolute_yd_line includes endzone width

    first_down_marker = (
        (line_of_scrimmage + df_focused.yardsToGo.values[0])
        if play_going_right
        else (line_of_scrimmage - df_focused.yardsToGo.values[0])
    )  # Calculate 1st down marker

    down = df_focused.down.values[0]
    quarter = df_focused.quarter.values[0]
    gameClock = df_focused.gameClock.values[0]
    playDescription = df_focused.playDescription.values[0]
    displayName = df_focused.displayName.values[0]
    tackle_frame_id = -1

    # Handle case where we have a really long Play Description and want to split it into two lines
    if len(playDescription.split(" ")) > 15 and len(playDescription) > 115:
        playDescription = (
            " ".join(playDescription.split(" ")[0:16])
            + "<br>"
            + " ".join(playDescription.split(" ")[16:])
        )

    print(
        f"Line of Scrimmage: {line_of_scrimmage}, First Down Marker: {first_down_marker}, Down: {down}, Quarter: {quarter}, Game Clock: {gameClock}, Play Description: {playDescription}"
    )

    # initialize plotly play and pause buttons for animation
    updatemenus_dict = [
        {
            "buttons": [
                {
                    "args": [
                        None,
                        {
                            "frame": {"duration": 100, "redraw": False},
                            "fromcurrent": True,
                            "transition": {"duration": 0},
                        },
                    ],
                    "label": "Play",
                    "method": "animate",
                },
                {
                    "args": [
                        [None],
                        {
                            "frame": {"duration": 0, "redraw": False},
                            "mode": "immediate",
                            "transition": {"duration": 0},
                        },
                    ],
                    "label": "Pause",
                    "method": "animate",
                },
            ],
            "direction": "left",
            "pad": {"r": 10, "t": 87},
            "showactive": False,
            "type": "buttons",
            "x": 0.1,
            "xanchor": "right",
            "y": 0,
            "yanchor": "top",
        }
    ]

    # initialize plotly slider to show frame position in animation
    sliders_dict = {
        "active": 0,
        "yanchor": "top",
        "xanchor": "left",
        "currentvalue": {
            "font": {"size": 20},
            "prefix": "Frame:",
            "visible": True,
            "xanchor": "right",
        },
        "transition": {"duration": 300, "easing": "cubic-in-out"},
        "pad": {"b": 10, "t": 50},
        "len": 0.9,
        "x": 0.1,
        "y": 0,
        "steps": [],
    }

    # Frame Info
    sorted_frame_list = df_focused.frameId.unique()
    sorted_frame_list.sort()

    frames = []
    for frameId in sorted_frame_list:
        data = []
        # Add Yardline Numbers to Field
        data.append(
            go.Scatter(
                x=np.arange(20, 110, 10),
                y=[5] * len(np.arange(20, 110, 10)),
                mode="text",
                text=list(
                    map(str, list(np.arange(20, 61, 10) - 10) + list(np.arange(40, 9, -10)))
                ),
                textfont_size=30,
                textfont_family="Courier New, monospace",
                textfont_color="#ffffff",
                showlegend=False,
                hoverinfo="none",
            )
        )
        data.append(
            go.Scatter(
                x=np.arange(20, 110, 10),
                y=[53.5 - 5] * len(np.arange(20, 110, 10)),
                mode="text",
                text=list(
                    map(str, list(np.arange(20, 61, 10) - 10) + list(np.arange(40, 9, -10)))
                ),
                textfont_size=30,
                textfont_family="Courier New, monospace",
                textfont_color="#ffffff",
                showlegend=False,
                hoverinfo="none",
            )
        )
        # Add line of scrimage
        data.append(
            go.Scatter(
                x=[line_of_scrimmage, line_of_scrimmage],
                y=[0, 53.5],
                line_dash="dash",
                line_color="blue",
                showlegend=False,
                hoverinfo="none",
            )
        )
        # Add First down line
        data.append(
            go.Scatter(
                x=[first_down_marker, first_down_marker],
                y=[0, 53.5],
                line_dash="dash",
                line_color="yellow",
                showlegend=False,
                hoverinfo="none",
            )
        )
        # Plot Players
        for club in df_focused.club.unique():
            plot_df = df_focused[
                (df_focused.club == club) & (df_focused.frameId == frameId)
            ].copy()
            if club != "football":
                hover_text_array = []
                for nflId in plot_df.nflId:
                    selected_player_df = plot_df[plot_df.nflId == nflId]
                    hover_text_array.append(
                        f"nflId:{selected_player_df['nflId'].values[0]}<br>displayName:{selected_player_df['displayName'].values[0]}"
                    )
                data.append(
                    go.Scatter(
                        x=plot_df["x"],
                        y=plot_df["y"],
                        mode="markers",
                        marker_color=colors[club],
                        marker_size=10,
                        name=club,
                        hovertext=hover_text_array,
                        hoverinfo="text",
                    )
                )
                if (
                    plot_df.event.values[0] == "tackle"
                    and club == plot_df.possessionTeam.values[0]
                ):
                    tackle_frame_id = frameId
                    ballcarrier_df = df_focused[
                        (df_focused.nflId == displayName)
                        & (df_focused.frameId == frameId)
                    ].copy()
                    data.append(
                        go.Scatter(
                            x=ballcarrier_df["x"],
                            y=ballcarrier_df["y"],
                            mode="markers",
                            marker_color=colors["tackle"],
                            marker_size=25,
                            name="tackle",
                            hovertext=["Tackle"],
                            hoverinfo="text",
                        )
                    )
            else:
                data.append(
                    go.Scatter(
                        x=plot_df["x"],
                        y=plot_df["y"],
                        mode="markers",
                        marker_color=colors[club],
                        marker_size=10,
                        name=club,
                        hoverinfo="none",
                    )
                )

        # add frame to slider
        slider_step = {
            "args": [
                [frameId],
                {
                    "frame": {"duration": 100, "redraw": False},
                    "mode": "immediate",
                    "transition": {"duration": 0},
                },
            ],
            "label": str(frameId),
            "method": "animate",
        }
        sliders_dict["steps"].append(slider_step)
        frames.append(go.Frame(data=data, name=str(frameId)))

    scale = 10
    layout = go.Layout(
        autosize=False,
        width=120 * scale,
        height=60 * scale,
        xaxis=dict(
            range=[0, 120],
            autorange=False,
            tickmode="array",
            tickvals=np.arange(10, 111, 5).tolist(),
            showticklabels=False,
        ),
        yaxis=dict(range=[0, 53.3], autorange=False, showgrid=False, showticklabels=False),
        plot_bgcolor="#00B140",
        # Create title and add play description at the bottom of the chart for better visual appeal
        title=f"GameId: {game_id}, PlayId: {play_id}<br>{gameClock} {quarter}Q, Tackled at Frame {tackle_frame_id}"
        + "<br>" * 19
        + f"{playDescription}",
        updatemenus=updatemenus_dict,
        sliders=[sliders_dict],
    )

    fig = go.Figure(data=frames[0]["data"], layout=layout, frames=frames[1:])

    # Create First Down Markers
    for y_val in [0, 53]:
        fig.add_annotation(
            x=first_down_marker,
            y=y_val,
            text=str(down),
            showarrow=False,
            font=dict(family="Courier New, monospace", size=16, color="black"),
            align="center",
            bordercolor="black",
            borderwidth=2,
            borderpad=4,
            bgcolor="#ff7f0e",
            opacity=1,
        )

    fig.show()

In [ ]:
visualize(2022110609,105)